In [ ]:
import geemap
import ee
import json
import os
import requests
import re
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import pandas as pd

import numpy as np
from glob import glob
import urllib.request
from matplotlib.pyplot import imshow
from PIL import Image,ImageDraw,ImageFont

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#initialize GEE using your Google Account
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()
    
    